In [7]:
import os
import subprocess
import pandas as pd

In [2]:
roi_window_file_processed = "/afs/bx.psu.edu/user/d/dzb5732/work/girirajan_lab/starrseq/data/roi.homer.window.bed"

# Get the HOMER motif file

In [3]:
homer_motif_database_url = "http://homer.ucsd.edu/homer/custom.motifs"
homer_motifs = "/afs/bx.psu.edu/user/d/dzb5732/work/girirajan_lab/starrseq/data/homer.motifs"

!wget -O $homer_motifs $homer_motif_database_url

--2021-12-20 11:18:50--  http://homer.ucsd.edu/homer/custom.motifs
Resolving homer.ucsd.edu (homer.ucsd.edu)... 169.228.63.226
Connecting to homer.ucsd.edu (homer.ucsd.edu)|169.228.63.226|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 194498 (190K)
Saving to: '/afs/bx.psu.edu/user/d/dzb5732/work/girirajan_lab/starrseq/data/homer.motifs'

/afs/bx.psu.edu/use 100%[===================>] 189.94K  1004KB/s    in 0.2s    

2021-12-20 11:18:50 (1004 KB/s) - '/afs/bx.psu.edu/user/d/dzb5732/work/girirajan_lab/starrseq/data/homer.motifs' saved [194498/194498]



# Create an intersection of genomic window file with the master enhancer list

This will give us genomic windows of length 500 which will have TF motifs

- Decide the fraction of intersection: start with 50 bp or 10% of the window must intersect with the enhancer region

In [4]:
genomic_window_file = "/data5/deepro/tf_bind_prediction/data/windows.bed"
roi_file = "/afs/bx.psu.edu/user/d/dzb5732/work/girirajan_lab/starrseq/data/master.homer.bed"
roi_window_file = "/afs/bx.psu.edu/user/d/dzb5732/work/girirajan_lab/starrseq/data/master.homer.window.bed"

!bedtools intersect -a $genomic_window_file -b $roi_file -f 0.1 -u -sorted > $roi_window_file

## Post process the roi window file to be compatible with homer

In [10]:
df_roi_raw = pd.read_csv(roi_window_file, sep="\t", header=None, usecols=[0,1,2])

In [12]:
def process(df_row):
    chrname = df_row[0]
    start = df_row[1]
    end = df_row[2]
    peak_name = f"{chrname}:{start}-{end}"
    irrelevant_col = 0
    strand = "."
    return pd.Series({"3":peak_name, "4":irrelevant_col, "5": strand}) 

In [18]:
df_roi_raw.merge(df_roi_raw.apply(process, axis=1), left_index=True, right_index=True).to_csv(roi_window_file_processed, index=False, header=None, sep="\t")

# Define the activity of each roi window based on the number of reads that align to that region
- Align filtered reads to get read counts
- Think about getting rid of covariates later

In [6]:
# Filtered read bam file and roi window file for both input and output coverage
input_bam_file = "/data5/deepro/starrseq/main_lib/filtered_libraries/Input_SeqReady.bam"
input_cov_out = "/data5/deepro/starrseq/main_lib/results/activity_prediction/data/input.cov.bed"
cc_bam_file = "/data5/deepro/starrseq/main_lib/filtered_libraries/CC.bam"
cc_cov_out = "/data5/deepro/starrseq/main_lib/results/activity_prediction/data/cc.cov.bed"
atf2_bam_file = "/data5/deepro/starrseq/main_lib/filtered_libraries/ATF2.bam"
atf2_cov_out = "/data5/deepro/starrseq/main_lib/results/activity_prediction/data/atf2.cov.bed"

In [3]:
!bash get_coverage.sh $roi_window_file_processed $input_bam_file $input_cov_out false

In [4]:
!bash get_coverage.sh $roi_window_file_processed $cc_bam_file $cc_cov_out false

In [7]:
!bash get_coverage.sh $roi_window_file_processed $atf2_bam_file $atf2_cov_out false

# Find the log odds score of all the motifs present in the HOMER database that might bind to each region

In [19]:
motif_location_output_dir = "/data5/deepro/starrseq/main_lib/results/activity_prediction/data/"
motif_location_output_file = "/data5/deepro/starrseq/main_lib/results/activity_prediction/data/motif_odds.tsv"
genome_fasta = "/data5/deepro/genomes/GRCh38_no_alt_analysis_set_GCA_000001405.15.fasta"

In [25]:
!findMotifsGenome.pl $roi_window_file_processed $genome_fasta $motif_location_output_dir -find $homer_motifs > $motif_location_output_file


	Position file = /afs/bx.psu.edu/user/d/dzb5732/work/girirajan_lab/starrseq/data/roi.homer.window.bed
	Genome = /data5/deepro/genomes/GRCh38_no_alt_analysis_set_GCA_000001405.15.fasta
	Output Directory = /data5/deepro/starrseq/main_lib/results/activity_prediction/data/
	Will find motif(s) in /afs/bx.psu.edu/user/d/dzb5732/work/girirajan_lab/starrseq/data/homer.motifs
	Using Custom Genome
	Peak/BED file conversion summary:
		BED/Header formatted lines: 946702
		peakfile formatted lines: 0

	Peak File Statistics:
		Total Peaks: 946702
		Redundant Peak IDs: 0
		Peaks lacking information: 0 (need at least 5 columns per peak)
		Peaks with misformatted coordinates: 0 (should be integer)
		Peaks with misformatted strand: 0 (should be either +/- or 0/1)

	Peak file looks good!

	Background files for 200 bp fragments found.
	Custom genome sequence file: /data5/deepro/genomes/GRCh38_no_alt_analysis_set_GCA_000001405.15.fasta

	Extracting sequences from file: /data5/deepro/genomes/GRCh38_no_alt_